## Instructions

1. Follow these instructions from `packages/grid/veilid/development.md` to build veilid docker containers:
   ```bash
   cd packages/grid/veilid && docker build -f veilid.dockerfile -t veilid:0.1 .
   ```
2. From within the `packages/grid/veilid` directory run the receiver docker container on port 4000:
   ```bash
   docker run -it -e DEV_MODE=True -p 4000:4000 -v $(pwd)/server:/app/server veilid:0.1
   ```
3. On a separate terminal tab/window, cd into `packages/grid/veilid` directory again and run the sender docker container on port 4001:
   ```bash
   docker run -it -e DEV_MODE=True -p 4001:4000 -v $(pwd)/server:/app/server veilid:0.1
   ```
4. Follow and run the below cells to test out sending large messages through Veilid. You may also use the **`Run All`** notebook function once the above two docker containers are up and running.

### 1. Set up imports

In [ ]:
# stdlib
from pprint import pprint
import time

# third party
import requests

### 2. Set up receiver

In [ ]:
RECEIVER_HOST = "localhost"
RECEIVER_PORT = 4000
RECEIVER_BASE_ADDRESS = f"http://{RECEIVER_HOST}:{RECEIVER_PORT}"

requests.post(f"{RECEIVER_BASE_ADDRESS}/generate_dht_key")
res = requests.get(f"{RECEIVER_BASE_ADDRESS}/retrieve_dht_key")
receiver_dht_key = res.json()["message"]
print(f"{'=' * 30}\n{receiver_dht_key}\n{'=' * 30}")

### 3. Set up sender

In [ ]:
SENDER_HOST = "localhost"
SENDER_PORT = 4001
SENDER_BASE_ADDRESS = f"http://{SENDER_HOST}:{SENDER_PORT}"

requests.post(f"{SENDER_BASE_ADDRESS}/generate_dht_key")
res = requests.get(f"{SENDER_BASE_ADDRESS}/retrieve_dht_key")
sender_dht_key = res.json()["message"]
print(f"{'=' * 30}\n{sender_dht_key}\n{'=' * 30}")

### 4. Declare function to send message of arbitrary size

In [ ]:
def send_ping(size_kb):
    size_bytes = size_kb * 1024
    message = "ping" * (size_bytes // 4)
    json_data = {
        "dht_key": receiver_dht_key,
        "message": message,
    }
    print(f"Sending message of size {len(message) // 1024} KB...")
    start = time.time()
    app_call = requests.post(f"{SENDER_BASE_ADDRESS}/app_call", json=json_data)
    end = time.time()
    time_taken = round(end - start, 2)
    response = app_call.json()["response"]
    print(f"[{time_taken}s] Response({len(response) // 1024} KB): {response[:100]}...")
    return time_taken

### 5. Send messages from 1 KB to 512 MB in size and benchmark them

In [ ]:
# Baseline tests (Tests with single chunk messages i.e. 1 KB to 32 KB)
benchmarks = {}

for message_size_kb in range(0, 6):  # Test from 1 KB to 32 KB
    message_size_kb = 2**message_size_kb
    benchmarks[message_size_kb] = send_ping(message_size_kb)

pprint(benchmarks)

In [ ]:
# Tests with smaller messages
for message_size_kb in range(5, 13):  # Test from 32 KB to 4 MB
    message_size_kb = 2**message_size_kb
    benchmarks[message_size_kb] = send_ping(message_size_kb)

pprint(benchmarks)

In [ ]:
# Tests with larger messages
for message_size_kb in range(12, 16):  # Test from 4 MB to 32 MB
    message_size_kb = 2**message_size_kb
    benchmarks[message_size_kb] = send_ping(message_size_kb)

pprint(benchmarks)

In [ ]:
# Tests with super large messages (currently not supported)
for message_size_kb in range(16, 20):  # Test from 64 MB to 512 MB
    message_size_kb = 2**message_size_kb
    benchmarks[message_size_kb] = send_ping(message_size_kb)

pprint(benchmarks)